# idea will be to get the dataset and train our model and track multiple info

In [2]:
import pandas as pd

import mlflow
from mlflow.models import infer_signature

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:

## set the tracking uri to the default port
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
## load the dataset

X,y = datasets.load_iris(return_X_y = True,)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

## just say these are the best ones for this project

params = {  "solver":'lbfgs',
            "max_iter":1000,
            "multi_class":"auto",
            "random_state":42,
            "penalty": "l2",
        }

## train the model

lr = LogisticRegression(**params)
lr  = lr.fit(X_train, y_train)

/Users/szanwar/Desktop/MLOPS/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [30]:
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
accuracy

1.0

In [31]:
## lets track the accuracy metric in MLFLOW
mlflow.set_tracking_uri("http://127.0.0.1:5000")

## new exp. for mlflow
mlflow.set_experiment("MLFlow quick start -- basic understanding")

## start the mlflow run
with mlflow.start_run():
    mlflow.log_params(params) 

    ## log_metric when you know the inddividual param to track
    ## since we had the dictionary of params: so log_params
    
    ## logging the accuracy metric, if more metrics: then log_metrics within dictionary
    mlflow.log_metric("accuracy", accuracy)

    ## setting a tag so that we know what was this metric for
    mlflow.set_tag("Training info: ", "LR model for the iris data")

    ## infer the model signature: -- schema of input and output are fixed
    signature = infer_signature(X_train, lr.predict(X_train))
            ## X_train is our input schema
            ## lr.predict(X_train) is our output schema
    
    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = 'tracking -- quick start mlflow'
    ) 

2025/01/19 12:21:59 INFO mlflow.tracking.fluent: Experiment with name 'MLFlow quick start -- basic understanding' does not exist. Creating a new experiment.


Successfully registered model 'tracking -- quick start mlflow'.
2025/01/19 12:22:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking -- quick start mlflow, version 1


🏃 View run flawless-steed-623 at: http://127.0.0.1:5000/#/experiments/379958097058336428/runs/7680548076e44377b3946dc03c79b049
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379958097058336428


Created version '1' of model 'tracking -- quick start mlflow'.


## new model with new params

In [32]:
params = {  "solver":'newton-cg',
            "max_iter":1000,
            "multi_class":"auto",
            "random_state":43
        }

## train the model

lr = LogisticRegression(**params)
lr  = lr.fit(X_train, y_train)

## prediction
y_pred = lr.predict(X_test)

## accuracy score
accuracy = accuracy_score(y_pred, y_test)
accuracy

/Users/szanwar/Desktop/MLOPS/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


1.0

In [33]:
## start the mlflow run
with mlflow.start_run():
    mlflow.log_params(params) 

    ## log_metric when you know the inddividual param to track
    ## since we had the dictionary of params: so log_params
    
    ## logging the accuracy metric, if more metrics: then log_metrics within dictionary
    mlflow.log_metric("accuracy", accuracy)

    ## setting a tag so that we know what was this metric for
    mlflow.set_tag("Training info: ", "LR model for the iris data")

    ## infer the model signature: -- schema of input and output are fixed
    signature = infer_signature(X_train, lr.predict(X_train))
            ## X_train is our input schema
            ## lr.predict(X_train) is our output schema
    
    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = 'tracking -- quick start mlflow'
    ) 

Registered model 'tracking -- quick start mlflow' already exists. Creating a new version of this model...
2025/01/19 12:39:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking -- quick start mlflow, version 2


🏃 View run dapper-grub-489 at: http://127.0.0.1:5000/#/experiments/379958097058336428/runs/bfd7dfa0beb447cbb1eed08e503ab0e7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379958097058336428


Created version '2' of model 'tracking -- quick start mlflow'.


## inferencing and validating my model

        -- when you go to the mlflow uri, theres a code given to validate the model in the artifacts section
        -- just copy that code and see how it works

In [34]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/bfd7dfa0beb447cbb1eed08e503ab0e7/iris_model'  
## same as: model_info.model_uri

## this is how the model .pkl got stored in mlruns: bfd7dfa0beb447cbb1eed08e503ab0e7
## its model-uri
# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      5.8,
      2.7,
      3.9,
      1.2
    ],
    [
      6.7,
      3.0,
      5.0,
      1.7
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      6.0,
      2.9,
      4.5,
      1.5
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      6.2,
      2.2,
      4.5,
      1.5
    ],
    [
      6.3,
      3.3,
      4.7,
      1.6
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      5.1,
      3.4,
      1.5,
      0.2
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      6.7,
      2.5,
      5.8,
      1.8
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      6.5,
      3.2,
      5.1,
      2.0
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      5.1,
      3.8,
      1.5,
      0.3
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      5.0,
      3.4,
      1.6,
      0.4
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      6.8,
      3.2,
      5.9,
      2.3
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      5.4,
      3.4,
      1.5,
      0.4
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      6.7,
      3.0,
      5.2,
      2.3
    ],
    [
      4.7,
      3.2,
      1.3,
      0.2
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      5.6,
      3.0,
      4.5,
      1.5
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      6.1,
      2.8,
      4.7,
      1.2
    ],
    [
      5.5,
      2.3,
      4.0,
      1.3
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      6.4,
      2.7,
      5.3,
      1.9
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      5.5,
      2.4,
      3.7,
      1.0
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      4.4,
      3.2,
      1.3,
      0.2
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      7.7,
      2.6,
      6.9,
      2.3
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([0, 2, 1, 2, 1, 1, 0, 1, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1,
       2, 2, 0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 1, 0, 2, 0, 0, 2,
       2, 0, 2, 1, 1, 1, 2, 2, 0, 2, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0,
       2, 2, 1, 0, 0, 1, 1, 0, 1, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 1, 1, 1,
       1, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 1, 0, 0, 0, 2, 0, 2, 1, 1, 1, 1,
       1, 0, 0, 1, 2, 2, 2, 2, 2, 0])

## since we got the output
        -- that means the model is working/validatin

### 2nd way of validating model
        -- loading the model as per python generic model

In [37]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

iris_features_name  = datasets.load_iris().feature_names
result  = pd.DataFrame(X_test, columns = iris_features_name)
result['actual_class']  = y_test
result['predicted_class'] = predictions

In [40]:
result.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,7.7,2.8,6.7,2.0,2,2
1,4.8,3.4,1.6,0.2,0,0
2,5.0,3.3,1.4,0.2,0,0
3,7.0,3.2,4.7,1.4,1,1
4,5.1,3.5,1.4,0.3,0,0


## Model Registry
        -- acts like a store: has all the models we run
        -- consist of set of apis and UI to collaboritively manage the full life-cycle of the MLFLow model. 
        -- helps in model versioning, model aliasing, model tagging and annotations


        -- so when you wrote the model_info: we wrote - model_register
        -- we should never do that, since we haven't validated the model
        -- so don't include model_register there and register only after its validated

## Example
        -- when you run this and then see the artifacts, you see its asking me to register the model

In [41]:
## start the mlflow run
with mlflow.start_run():
    mlflow.log_params(params) 

    ## log_metric when you know the inddividual param to track
    ## since we had the dictionary of params: so log_params
    
    ## logging the accuracy metric, if more metrics: then log_metrics within dictionary
    mlflow.log_metric("accuracy", 1.0) ## giving a har coded value 

    ## setting a tag so that we know what was this metric for
    mlflow.set_tag("Training info: ", "LR model for the iris data")

    ## infer the model signature: -- schema of input and output are fixed
    signature = infer_signature(X_train, lr.predict(X_train))
            ## X_train is our input schema
            ## lr.predict(X_train) is our output schema
    
    ## log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        ## registered_model_name = 'tracking -- quick start mlflow'
    ) 

🏃 View run zealous-seal-278 at: http://127.0.0.1:5000/#/experiments/379958097058336428/runs/d82ee277b93b4a41958884fb98ffe797
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379958097058336428


## so we didn't register because?
    -- run all the models
    -- compare the results in MLFlow
    -- and see whats gonna work best 
    -- and then register that model

## so now what after best model is registered

        -- lets infer the model which we thought is the best
        -- and lets use that as the final model

In [4]:
import mlflow.sklearn

model_name = "tracking -- quick start mlflow"
model_version = 3  ## since the version 3 one was the best we selected as best one

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)
model

LogisticRegression(max_iter=1000, multi_class='auto', random_state=43,
                   solver='newton-cg')

In [7]:
y_pred_new = model.predict(X_test)

accuracy_score(y_test, y_pred_new)

1.0